In [18]:
import json
import pandas as pd
import numpy as np

In [83]:
def flatten_outer (data):
    """
    Description: This is upper level function which has a list that stores each row values dictionary
    Parameters: data, accepts each record of a json file
    """
    full_list = []
    def flatten_inner(sub_data,first_level_key='',index=0,tot_len=0):
        
        """
        Description: This function iterates through each and every key value pair in a JSON record. If any key has a list of items
                     then this function will preserve the structure of the data and creates a row value as a list for each column. 
                     In this way, there will be no change in the number of rows of a dataframe, and it will be easy to perform some 
                     further cleaning and transformation tasks
        Parameters: sub_data - for each key which has a dictionary or list value will be passed again to this function as a part of recursion.
                                This parameter contains the dict or list value
                    first_level_key - first level key in the json file
                    index - this index value is used to send the item position if a value is a list, otherwise it will be 0
                    tot_len - if any key has a list nested value, then the length of list nested value will be passed in here. This value will be used to created a placeholder list
        """
        for k,v in sub_data.items():
            full_key = first_level_key+'.'+k if first_level_key !='' else k
            if isinstance(v, dict): 
                flatten_inner(v, full_key)
                
            elif isinstance(v, list):
                for i in range(0, len(v)): 
                    if (isinstance(v[i], dict)):
                        flatten_inner(v[i], full_key,index=i, tot_len=len(v))
                    else: 
                        value_list[full_key] = v
            else:

                if full_key in value_list.keys():
                    placeholder_list = value_list[full_key]
                    placeholder_list[index] = v
                    value_list[full_key] = placeholder_list
                else:
                    if index == 0:
                        if tot_len == 0:
                            value_list[full_key] = v
                        else:
                            placeholder_list = [None]*tot_len
                            placeholder_list[0] = v
                            value_list[full_key] = placeholder_list
                    else:
                        
                        dif = tot_len - index - 1
                        placeholder_list = [None] * index
                        placeholder_list.append(v)
                        placeholder_list = placeholder_list + [None] * dif
                        value_list[full_key] = placeholder_list
                
        return value_list
        
    for row in data:
        value_list = dict() #creating a value_list to store key value pairs(column values) for each record
        cv =  flatten_inner(row)
        full_list.append(cv)
    
    return full_list

In [84]:
def df_create_clean(full_list):
    df = pd.DataFrame(full_list)
    df = df.where(pd.notnull(df),None)
    cols = df.columns
    for col in cols:
        df[col] = df[col].apply(lambda x: None if (isinstance(x,list) and len(x)==0) else x)
        df[col] = df[col].apply(lambda x: x[0] if (isinstance(x,list) and len(x)==1) else x)
    return df


In [91]:
data = [{'state': 'Florida',
          'shortname': 'FL',
          'info': {
               'governor': 'Rick Scott'
          },
          'counties': [{'name': 'Dade', 'population': 12345,'check':'YesYes'},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
         {'state': 'Ohio',
          'shortname': 'OH',
          'info': {
               'governor': 'John Kasich'
          },
          'counties': [{'name': 'Summit', 'population': 1234},
                       {'name': 'Cuyahoga', 'population': 1337},
                      {'name': 'Cuyahoga2', 'population': 1331,'check':'YesYes'}]}]

In [92]:
ls = flatten_outer(data)
f_df = df_create_clean(ls)
    

finkey is: state
finkey value is: Florida
finkey is: shortname
finkey value is: FL
finkey is: info.governor
finkey value is: Rick Scott
finkey is: counties.name
finkey value is: ['Dade', None, None]
finkey is: counties.population
finkey value is: [12345, None, None]
finkey is: counties.check
finkey value is: ['YesYes', None, None]
finkey is: counties.name
finkey value is: ['Dade', 'Broward', None]
finkey is: counties.population
finkey value is: [12345, 40000, None]
finkey is: counties.name
finkey value is: ['Dade', 'Broward', 'Palm Beach']
finkey is: counties.population
finkey value is: [12345, 40000, 60000]
finkey is: state
finkey value is: Ohio
finkey is: shortname
finkey value is: OH
finkey is: info.governor
finkey value is: John Kasich
finkey is: counties.name
finkey value is: ['Summit', None, None]
finkey is: counties.population
finkey value is: [1234, None, None]
finkey is: counties.name
finkey value is: ['Summit', 'Cuyahoga', None]
finkey is: counties.population
finkey value is:

In [93]:
f_df

,state,shortname,info.governor,counties.name,counties.population,counties.check
0,Florida,FL,Rick Scott,"[Dade, Broward, Palm Beach]","[12345, 40000, 60000]","[YesYes, None, None]"
1,Ohio,OH,John Kasich,"[Summit, Cuyahoga, Cuyahoga2]","[1234, 1337, 1331]","[None, None, YesYes]"
